In [ ]:
""" Pip Install """

%pip install requests bs4

In [1]:
""" Imports """

import requests
from bs4 import BeautifulSoup
from urllib.parse import unquote, urlparse

In [2]:
""" Aux func """

def print_each(list_of_links):
    """ print each """
    for link in list_of_links:
        print(link)

def print_size(list_of_links, link = ''):
    """ print size """
    print('size:', '\t', len(list_of_links), '\t', link)

def dedupe_and_sort(links):
    """ filter and sort links """
    links = list(set(links)) # deduping
    links.sort() # sorting
    return links

In [ ]:
""" Scraping """

base_link = 'https://lgbtqia.fandom.com/wiki/'

links = ['', 'Special:AllPages', 'Special:AllPages?from=LGBTQA',
         'Special:Categories', 'Category:Flags', 'Category:Stubs',]

links = [base_link + link for link in links]

allHyperLinks = {}


def clean_found_hlinks(all_a):
    # print(len(all_a))
    current_hyperlinks = []
    for link in all_a:
        # print(link)
        got_href = link.get('href')
        if got_href is not None:
            # print(got_href)
            got_href = got_href.lstrip('/')
            got_href = got_href.lower()
            full_link = got_href
            # print(got_href)
            if got_href.startswith('wiki/'):
                full_link = 'https://lgbtqia.fandom.com/' + got_href
            if full_link.startswith(base_link):
                # print(full_link)
                current_hyperlinks.append(full_link)
    filtered_links = dedupe_and_sort(current_hyperlinks)
    return filtered_links

def get_hyperlinks(url):
    """ get hyperlinks """
    all_a = soup.find_all('a')
    cleaned_hlinks = clean_found_hlinks(all_a)
    return cleaned_hlinks

def get_hyperlinks_from_list_of_links(links):
    """ get hyperlinks from list of links """
    allHyperLinks = []
    for link in links:
        current_link_hyperlinks = get_hyperlinks(link)
        print_size(current_link_hyperlinks, link)
        allHyperLinks.append(current_link_hyperlinks)
    return allHyperLinks

allHyperLinks = get_hyperlinks_from_list_of_links(links)

def get_soup(link):
    """ parse link """
    page = requests.get(link)
    if page.status_code != 200:
        return []
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup
    
    

print('collected links once')

In [ ]:
""" Check items that are in all lists """

def get_common_links(all_links):
    """ get common links """
    common_links = set(all_links[0])
    for links in all_links[1:]:
        common_links.intersection_update(links)
    return common_links

common_links = get_common_links(allHyperLinks)

print_size(common_links, 'common links')

print(common_links)

In [ ]:
""" Filtering Links """

print('size:', len(allHyperLinks))
allHyperLinks = dedupe_and_sort(allHyperLinks)
print('size:', len(allHyperLinks))


In [ ]:
""" Second Iteration """

allHyperLinks += get_hyperlinks_from_list_of_links(allHyperLinks)


In [ ]:
print('size:', len(allHyperLinks))
filteredAllHyperLinks = dedupe_and_sort(allHyperLinks).copy()
print('size:', len(allHyperLinks))

In [ ]:
""" More filtering """

def link_404(link):
    """ remove 404 links """
    page = requests.get(link)
    return page.status_code == 404

def link_question(link):
    """ remove question links """
    return '?' in link

def link_hashtag(link):
    """ remove hashtag links """
    return '#' in link

def filtering_links(links):
    """ filtering links """
    for link in links:
        print(link)
        if link_question(link) or link_404(link) or link_hashtag(link):
            print('removing:', link)
            links.remove(link)
    return links



print_size(filteredAllHyperLinks)
# filteredAllHyperLinks = filtering_links(filteredAllHyperLinks)
print_size(filteredAllHyperLinks)

In [ ]:
unexisting_pages = [
    'https://lgbtqia.fandom.com/wiki/2-spirit',
    'https://lgbtqia.fandom.com/wiki/ace_lesbian',
    'https://lgbtqia.fandom.com/wiki/equality_act',
    'https://lgbtqia.fandom.com/wiki/adonic',
    'https://lgbtqia.fandom.com/wiki/equality_act_2010',
]

for link in unexisting_pages:
    page = requests.get(link)
    print(page.status_code)
    